In [ ]:
# Importing necessary libraries and dependencies
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
from zipfile import ZipFile
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import power_transform
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix




import logging
import warnings
import joblib

In [159]:
zip_path = f"playground-series-s5e11.zip"


In [160]:
def extract_datasets(path):
    """
    reads the target dataset(s) from the listed files
    
    parameters:
        ZipFile(path, "r"): the 'path' variable takes in the zip file and processes
        it using the with ZipFile() method

        open(): opens the selected file name from the zipfile
        read_csv(): is a pandas method to read a csv file, `index_col=0` removes unwanted 


    returns:
        datasets of the selected data (i.e train.csv, test.csv) as train_dt and test_dt respectively
    
    """
    with ZipFile(path, "r") as zip_file:
        with zip_file.open("train.csv") as a:
            train_dt = pd.read_csv(a)
        with zip_file.open("test.csv") as b:
            test_dt = pd.read_csv(b)

        return train_dt, test_dt




In [161]:
train_df, test_df = extract_datasets(zip_path)
train_df, test_df = train_df.copy(), test_df.copy()


In [162]:
def num_cat_split(dataset):

    """
    splits the data into numerical and categorical columns for further analysis
    
    parameters:
        select_dtypes: used to select dataset of specific type (i.e number, object, float, int e.t.c)
        columns: lists out the column names for selected items/features
        
    returns:
        two variables (num_features and cat_features) containing the numerical columns and categorical columns respectively. 
    
    """

    num_features = dataset.select_dtypes(include=["number"]).columns
    cat_features = dataset.select_dtypes(include=["object"]).columns

    return num_features, cat_features


num_features, cat_features = num_cat_split(train_df)

In [163]:
# Power transform skewed columns
power_transform(train_df[["annual_income", "debt_to_income_ratio"]], method='yeo-johnson')
power_transform(test_df[["annual_income", "debt_to_income_ratio"]], method='yeo-johnson')
train_df.head()

# train_df = train_df.drop(columns=["grade_subgrade"])
# test_df = test_df.drop(columns=["grade_subgrade"])





,id,annual_income,debt_to_income_ratio,credit_score,loan_amount,interest_rate,gender,marital_status,education_level,employment_status,loan_purpose,grade_subgrade,loan_paid_back
0,0,29367.99,0.084,736,2528.42,13.67,Female,Single,High School,Self-employed,Other,C3,1.0
1,1,22108.02,0.166,636,4593.10,12.92,Male,Married,Master's,Employed,Debt consolidation,D3,0.0
2,2,49566.20,0.097,694,17005.15,9.76,Male,Single,High School,Employed,Debt consolidation,C5,1.0
3,3,46858.25,0.065,533,4682.48,16.10,Female,Single,High School,Employed,Debt consolidation,F1,1.0
4,4,25496.70,0.053,665,12184.43,10.21,Male,Married,High School,Employed,Other,D1,1.0


In [164]:
test_df.head()

,id,annual_income,debt_to_income_ratio,credit_score,loan_amount,interest_rate,gender,marital_status,education_level,employment_status,loan_purpose,grade_subgrade
0,593994,28781.05,0.049,626,11461.42,14.73,Female,Single,High School,Employed,Other,D5
1,593995,46626.39,0.093,732,15492.25,12.85,Female,Married,Master's,Employed,Other,C1
2,593996,54954.89,0.367,611,3796.41,13.29,Male,Single,Bachelor's,Employed,Debt consolidation,D1
3,593997,25644.63,0.110,671,6574.30,9.57,Female,Single,Bachelor's,Employed,Debt consolidation,C3
4,593998,25169.64,0.081,688,17696.89,12.80,Female,Married,PhD,Employed,Business,C1


In [165]:
def convert_dtype(df):
    df = df.copy()
    df = df.astype({col: "category" for col in df.select_dtypes("object").columns})
    return df


train_df_preproc = convert_dtype(train_df)


In [166]:
test_df_preproc = convert_dtype(test_df)

In [167]:
train_df_preproc.head()

,id,annual_income,debt_to_income_ratio,credit_score,loan_amount,interest_rate,gender,marital_status,education_level,employment_status,loan_purpose,grade_subgrade,loan_paid_back
0,0,29367.99,0.084,736,2528.42,13.67,Female,Single,High School,Self-employed,Other,C3,1.0
1,1,22108.02,0.166,636,4593.10,12.92,Male,Married,Master's,Employed,Debt consolidation,D3,0.0
2,2,49566.20,0.097,694,17005.15,9.76,Male,Single,High School,Employed,Debt consolidation,C5,1.0
3,3,46858.25,0.065,533,4682.48,16.10,Female,Single,High School,Employed,Debt consolidation,F1,1.0
4,4,25496.70,0.053,665,12184.43,10.21,Male,Married,High School,Employed,Other,D1,1.0


In [168]:
test_df_preproc.head()

,id,annual_income,debt_to_income_ratio,credit_score,loan_amount,interest_rate,gender,marital_status,education_level,employment_status,loan_purpose,grade_subgrade
0,593994,28781.05,0.049,626,11461.42,14.73,Female,Single,High School,Employed,Other,D5
1,593995,46626.39,0.093,732,15492.25,12.85,Female,Married,Master's,Employed,Other,C1
2,593996,54954.89,0.367,611,3796.41,13.29,Male,Single,Bachelor's,Employed,Debt consolidation,D1
3,593997,25644.63,0.110,671,6574.30,9.57,Female,Single,Bachelor's,Employed,Debt consolidation,C3
4,593998,25169.64,0.081,688,17696.89,12.80,Female,Married,PhD,Employed,Business,C1


In [169]:
def encode_categorical(tr_df, ts_df, cols):
    encoders = {}
    for col in cols:
        le = LabelEncoder()
        tr_df[col] = le.fit_transform(tr_df[col].astype(str))
        ts_df[col] = le.transform(ts_df[col].astype(str).map(
            lambda x: x if x in le.classes_ else le.classes_[0]
        ))
        encoders[col] = le
    return tr_df, ts_df, encoders

In [170]:
from sklearn.preprocessing import StandardScaler



train, test, encoders = encode_categorical(train_df_preproc, test_df_preproc, cat_features)




In [171]:
train.head()

,id,annual_income,debt_to_income_ratio,credit_score,loan_amount,interest_rate,gender,marital_status,education_level,employment_status,loan_purpose,grade_subgrade,loan_paid_back
0,0,29367.99,0.084,736,2528.42,13.67,0,2,1,2,6,12,1.0
1,1,22108.02,0.166,636,4593.10,12.92,1,1,2,0,2,17,0.0
2,2,49566.20,0.097,694,17005.15,9.76,1,2,1,0,2,14,1.0
3,3,46858.25,0.065,533,4682.48,16.10,0,2,1,0,2,25,1.0
4,4,25496.70,0.053,665,12184.43,10.21,1,1,1,0,6,15,1.0


In [172]:
test.head()

,id,annual_income,debt_to_income_ratio,credit_score,loan_amount,interest_rate,gender,marital_status,education_level,employment_status,loan_purpose,grade_subgrade
0,593994,28781.05,0.049,626,11461.42,14.73,0,2,1,0,6,19
1,593995,46626.39,0.093,732,15492.25,12.85,0,1,2,0,6,10
2,593996,54954.89,0.367,611,3796.41,13.29,1,2,0,0,2,15
3,593997,25644.63,0.110,671,6574.30,9.57,0,2,0,0,2,12
4,593998,25169.64,0.081,688,17696.89,12.80,0,1,4,0,0,10


In [173]:
from sklearn.model_selection import train_test_split

# Define X and y
X = train.drop(columns=["loan_paid_back"])
y = train["loan_paid_back"]

scaler = StandardScaler()

num_features = num_features.drop('loan_paid_back', errors='ignore')

X[num_features] = scaler.fit_transform(X[num_features])


# Split into train/test (if not already split)
x_train, x_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


In [174]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Initialize and train model
log_reg = LogisticRegression(max_iter=1000, random_state=42)
log_reg.fit(x_train, y_train)

# Predict
y_pred = log_reg.predict(x_val)

# Evaluate
print("=== Logistic Regression Baseline ===")
print(f"Accuracy: {accuracy_score(y_val, y_pred):.4f}\n")
print("Classification Report:")
print(classification_report(y_val, y_pred))
print("\nConfusion Matrix:")
print(confusion_matrix(y_val, y_pred))


=== Logistic Regression Baseline ===
Accuracy: 0.8905

Classification Report:
              precision    recall  f1-score   support

         0.0       0.80      0.60      0.69     23900
         1.0       0.91      0.96      0.93     94899

    accuracy                           0.89    118799
   macro avg       0.86      0.78      0.81    118799
weighted avg       0.89      0.89      0.88    118799


Confusion Matrix:
[[14397  9503]
 [ 3503 91396]]


In [175]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Initialize and train Random Forest
rf = RandomForestClassifier(
    n_estimators=100,
    max_depth=10,
    random_state=42,
    class_weight={0: 3, 1: 1}  
)
rf.fit(x_train, y_train)

# Predict
y_pred_rf = rf.predict(x_val)

# Evaluate
print("=== Random Forest ===")
print(f"Accuracy: {accuracy_score(y_val, y_pred_rf):.4f}\n")
print("Classification Report:")
print(classification_report(y_val, y_pred_rf))
print("\nConfusion Matrix:")
print(confusion_matrix(y_val, y_pred_rf))

=== Random Forest ===
Accuracy: 0.8817

Classification Report:
              precision    recall  f1-score   support

         0.0       0.69      0.74      0.72     23900
         1.0       0.93      0.92      0.93     94899

    accuracy                           0.88    118799
   macro avg       0.81      0.83      0.82    118799
weighted avg       0.88      0.88      0.88    118799


Confusion Matrix:
[[17638  6262]
 [ 7793 87106]]


In [ ]:
# ===== IMPORTS =====
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score, accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.model_selection import RandomizedSearchCV

# ===== PHASE 1: ESTABLISH BASELINES =====
print("=== PHASE 1: ESTABLISH BASELINES ===")

# Your existing models
log_reg = LogisticRegression(max_iter=1000, random_state=42)
log_reg.fit(x_train, y_train)

rf = RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced')
rf.fit(x_train, y_train)

# Get probabilities and AUC scores
y_pred_proba_lr = log_reg.predict_proba(x_val)[:, 1]
y_pred_proba_rf = rf.predict_proba(x_val)[:, 1]

lr_auc = roc_auc_score(y_val, y_pred_proba_lr)
rf_auc = roc_auc_score(y_val, y_pred_proba_rf)

print(f"Logistic Regression AUC: {lr_auc:.4f}")
print(f"Random Forest AUC: {rf_auc:.4f}")
best_baseline = max(lr_auc, rf_auc)
best_model_name = "Logistic Regression" if lr_auc > rf_auc else "Random Forest"
print(f"Best baseline: {best_model_name} ({best_baseline:.4f})")

# ===== PHASE 2: GRADIENT BOOSTING =====
print("\n=== PHASE 2: GRADIENT BOOSTING ===")

# LightGBM
lgbm = LGBMClassifier(n_estimators=1000, random_state=42, verbose=-1)
lgbm.fit(x_train, y_train)
lgbm_auc = roc_auc_score(y_val, lgbm.predict_proba(x_val)[:, 1])
print(f"LightGBM AUC: {lgbm_auc:.4f}")

# XGBoost
xgb = XGBClassifier(n_estimators=1000, random_state=42, eval_metric='logloss')
xgb.fit(x_train, y_train)
xgb_auc = roc_auc_score(y_val, xgb.predict_proba(x_val)[:, 1])
print(f"XGBoost AUC: {xgb_auc:.4f}")

# Identify best model so far
models_auc = {
    'Logistic Regression': lr_auc,
    'Random Forest': rf_auc,
    'LightGBM': lgbm_auc,
    'XGBoost': xgb_auc
}
best_model_name_phase2 = max(models_auc, key=models_auc.get)
best_auc_phase2 = models_auc[best_model_name_phase2]
print(f"Best model after Phase 2: {best_model_name_phase2} ({best_auc_phase2:.4f})")

# ===== PHASE 3: SIMPLE ENSEMBLES =====
print("\n=== PHASE 3: SIMPLE ENSEMBLES ===")

# Get probabilities from all models
probas = {
    'lr': y_pred_proba_lr,
    'rf': y_pred_proba_rf,
    'lgbm': lgbm.predict_proba(x_val)[:, 1],
    'xgb': xgb.predict_proba(x_val)[:, 1]
}

# Equal weight ensemble
equal_ensemble = sum(probas.values()) / len(probas)
equal_auc = roc_auc_score(y_val, equal_ensemble)
print(f"Equal Weight Ensemble AUC: {equal_auc:.4f}")

# Weighted ensemble (favor best performers)
weights = {
    'lr': 0.2 if lr_auc > 0.8 else 0.1,
    'rf': 0.2 if rf_auc > 0.8 else 0.1,
    'lgbm': 0.3,
    'xgb': 0.3
}
weighted_ensemble = (
    probas['lr'] * weights['lr'] +
    probas['rf'] * weights['rf'] +
    probas['lgbm'] * weights['lgbm'] +
    probas['xgb'] * weights['xgb']
)
weighted_auc = roc_auc_score(y_val, weighted_ensemble)
print(f"Weighted Ensemble AUC: {weighted_auc:.4f}")

# ===== PHASE 4: FEATURE ENGINEERING =====
print("\n=== PHASE 4: FEATURE ENGINEERING ===")

def create_features(df):
    df = df.copy()
    
    # Create interaction features
    numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns
    
    # Simple ratio features (avoid division by zero)
    for col in numeric_cols:
        if 'income' in col.lower() or 'amount' in col.lower():
            for other_col in numeric_cols:
                if col != other_col and ('age' in other_col.lower() or 'term' in other_col.lower()):
                    df[f'{col}_ratio_{other_col}'] = df[col] / (df[other_col] + 1)
    
    # Polynomial features for important numeric columns
    for col in numeric_cols[:3]:  # Top 3 numeric columns
        df[f'{col}_squared'] = df[col] ** 2
        df[f'{col}_log'] = np.log1p(np.abs(df[col]))
    
    return df

# Apply feature engineering
x_train_eng = create_features(x_train)

x_val_eng = create_features(x_val)

print(f"Original features: {x_train.shape[1]}")
print(f"Engineered features: {x_train_eng.shape[1]}")

# Retrain best model on engineered features
if best_model_name_phase2 == "Logistic Regression":
    best_model = LogisticRegression(max_iter=1000, random_state=42)
elif best_model_name_phase2 == "Random Forest":
    best_model = RandomForestClassifier(n_estimators=100, random_state=42)
elif best_model_name_phase2 == "LightGBM":
    best_model = LGBMClassifier(n_estimators=1000, random_state=42, verbose=-1)
else:  # XGBoost
    best_model = XGBClassifier(n_estimators=1000, random_state=42)

best_model.fit(x_train_eng, y_train)
eng_auc = roc_auc_score(y_val, best_model.predict_proba(x_val_eng)[:, 1])
print(f"Best model + feature engineering AUC: {eng_auc:.4f}")

# ===== PHASE 5: HYPERPARAMETER TUNING =====
print("\n=== PHASE 5: HYPERPARAMETER TUNING ===")

# Only tune if we have time and it's worth it
if eng_auc > 0.85:  # Only tune if model is decent
    if best_model_name_phase2 in ['LightGBM', 'XGBoost']:
        param_dist = {
            'n_estimators': [500, 1000, 1500],
            'max_depth': [3, 6, 9],
            'learning_rate': [0.01, 0.05, 0.1]
        }
        
        best_tuned = RandomizedSearchCV(
            best_model, param_dist, n_iter=5, 
            scoring='roc_auc', cv=3, random_state=42, n_jobs=-1
        )
        best_tuned.fit(x_train_eng, y_train)
        
        tuned_auc = roc_auc_score(y_val, best_tuned.predict_proba(x_val_eng)[:, 1])
        print(f"Tuned {best_model_name_phase2} AUC: {tuned_auc:.4f}")
        print(f"Best params: {best_tuned.best_params_}")
    else:
        print("Skipping tuning for linear models (usually less gain)")
else:
    print("Skipping tuning - focus on better features/models first")

# ===== PHASE 6: FINAL EVALUATION =====
print("\n=== PHASE 6: FINAL RESULTS ===")

# Compare all approaches
final_results = {
    'Logistic Regression': lr_auc,
    'Random Forest': rf_auc,
    'LightGBM': lgbm_auc,
    'XGBoost': xgb_auc,
    'Equal Ensemble': equal_auc,
    'Weighted Ensemble': weighted_auc,
    'Feature Engineering': eng_auc
}

# Add tuned model if we did tuning
if 'tuned_auc' in locals():
    final_results['Tuned Model'] = tuned_auc

for model, auc in sorted(final_results.items(), key=lambda x: x[1], reverse=True):
    improvement = auc - best_baseline
    print(f"{model:25}: {auc:.4f} ({improvement:+.4f})")

best_final_model = max(final_results, key=final_results.get)
print(f"\n🎯 BEST FINAL APPROACH: {best_final_model}")
print(f"📈 Improvement over baseline: {final_results[best_final_model] - best_baseline:.4f}")

=== PHASE 1: ESTABLISH BASELINES ===
Logistic Regression AUC: 0.8902
Random Forest AUC: 0.9074
Best baseline: Random Forest (0.9074)

=== PHASE 2: GRADIENT BOOSTING ===
LightGBM AUC: 0.9209
XGBoost AUC: 0.9108
Best model after Phase 2: LightGBM (0.9209)

=== PHASE 3: SIMPLE ENSEMBLES ===
Equal Weight Ensemble AUC: 0.9151
Weighted Ensemble AUC: 0.9165

=== PHASE 4: FEATURE ENGINEERING ===
Original features: 12
Engineered features: 18
Best model + feature engineering AUC: 0.9215

=== PHASE 5: HYPERPARAMETER TUNING ===
Tuned LightGBM AUC: 0.9225
Best params: {'n_estimators': 1000, 'max_depth': 6, 'learning_rate': 0.05}

=== PHASE 6: FINAL RESULTS ===
Tuned Model              : 0.9225 (+0.0151)
Feature Engineering      : 0.9215 (+0.0142)
LightGBM                 : 0.9209 (+0.0135)
Weighted Ensemble        : 0.9165 (+0.0091)
Equal Ensemble           : 0.9151 (+0.0077)
XGBoost                  : 0.9108 (+0.0035)
Random Forest            : 0.9074 (+0.0000)
Logistic Regression      : 0.8902 (-

In [197]:
test_df.head()

,id,annual_income,debt_to_income_ratio,credit_score,loan_amount,interest_rate,gender,marital_status,education_level,employment_status,loan_purpose,grade_subgrade
0,-1.732058,-1.804962,-249.774466,-12.507359,-2.168832,-8.919472,Female,Single,High School,Employed,Other,D5
1,-1.732058,-1.804962,-113.319479,-12.506737,-2.168832,-9.151343,Female,Married,Master's,Employed,Other,C1
2,-1.732058,-1.804962,736.422939,-12.507448,-2.168832,-9.097075,Male,Single,Bachelor's,Employed,Debt consolidation,D1
3,-1.732058,-1.804962,-60.598234,-12.507095,-2.168832,-9.555883,Female,Single,Bachelor's,Employed,Debt consolidation,C3
4,-1.732058,-1.804962,-150.534476,-12.506995,-2.168832,-9.157510,Female,Married,PhD,Employed,Business,C1


In [191]:
power_transform(test_df[["annual_income", "debt_to_income_ratio"]], method='yeo-johnson')

array([[-0.69818506, -1.42654025],
       [ 0.1470805 , -0.24477471],
       [ 0.45572121,  2.10926188],
       ...,
       [-1.22938394, -0.17999054],
       [-0.41080032, -0.22298801],
       [ 0.10722231,  0.24618782]], shape=(254569, 2))

In [195]:
print("Scaling test data...")
test_df[num_features] = scaler.transform(test_df[num_features])

Scaling test data...


In [198]:
test_df.shape

(254569, 12)

In [200]:
x_test_eng.shape

(254569, 18)

In [196]:
test_df.head()

,id,annual_income,debt_to_income_ratio,credit_score,loan_amount,interest_rate,gender,marital_status,education_level,employment_status,loan_purpose,grade_subgrade
0,-1.732058,-1.804962,-249.774466,-12.507359,-2.168832,-8.919472,Female,Single,High School,Employed,Other,D5
1,-1.732058,-1.804962,-113.319479,-12.506737,-2.168832,-9.151343,Female,Married,Master's,Employed,Other,C1
2,-1.732058,-1.804962,736.422939,-12.507448,-2.168832,-9.097075,Male,Single,Bachelor's,Employed,Debt consolidation,D1
3,-1.732058,-1.804962,-60.598234,-12.507095,-2.168832,-9.555883,Female,Single,Bachelor's,Employed,Debt consolidation,C3
4,-1.732058,-1.804962,-150.534476,-12.506995,-2.168832,-9.157510,Female,Married,PhD,Employed,Business,C1


In [193]:
x_test_eng = create_features(test_df)

In [194]:
x_test_eng.head()

,id,annual_income,debt_to_income_ratio,credit_score,loan_amount,interest_rate,gender,marital_status,education_level,employment_status,loan_purpose,grade_subgrade,id_squared,id_log,annual_income_squared,annual_income_log,debt_to_income_ratio_squared,debt_to_income_ratio_log
0,-1.732038,-1.804922,-17.007139,-12.303256,-2.168593,-5.562493,Female,Single,High School,Employed,Other,D5,2.999955,1.005048,3.257743,1.031376,289.242768,2.890768
1,-1.732038,-1.804897,-7.649984,-12.268750,-2.168509,-6.028311,Female,Married,Master's,Employed,Other,C1,2.999955,1.005048,3.257653,1.031367,58.522248,2.157557
2,-1.732038,-1.804885,50.619574,-12.308139,-2.168753,-5.919289,Male,Single,Bachelor's,Employed,Debt consolidation,D1,2.999955,1.005048,3.257611,1.031363,2562.341269,3.943901
3,-1.732038,-1.804926,-4.034719,-12.288607,-2.168695,-6.841015,Female,Single,Bachelor's,Employed,Debt consolidation,C3,2.999955,1.005048,3.257759,1.031377,16.278957,1.616358
4,-1.732038,-1.804927,-10.201935,-12.283073,-2.168463,-6.040700,Female,Married,PhD,Employed,Business,C1,2.999955,1.005048,3.257762,1.031378,104.079477,2.416087


In [181]:
print(x_test_eng.dtypes.value_counts())
print(x_test_eng.select_dtypes('object').columns)
print(x_test_eng.shape)


float64    12
int64       6
Name: count, dtype: int64
Index([], dtype='object')
(254569, 18)


In [182]:
# Use your best model from Phase 5 (the tuned LightGBM)
final_predictions = best_tuned.predict_proba(x_test_eng)[:, 1]  # Probability of class 1

print(f"Generated {len(final_predictions)} predictions")
print(f"Prediction range: {final_predictions.min():.3f} to {final_predictions.max():.3f}")
print(f"Mean prediction: {final_predictions.mean():.3f}")

Generated 254569 predictions
Prediction range: 0.000 to 1.000
Mean prediction: 0.803


In [183]:
submission = pd.DataFrame({
    'id': x_test_eng['id'],
    'prediction': final_predictions
})

In [184]:
# Save in the correct format (usually CSV)
submission_file = 'submission.csv'
submission.to_csv(submission_file, index=False)

print(f"✅ Submission file saved as: {submission_file}")
print("📊 Submission stats:")
print(f"   - Shape: {submission.shape}")
print(f"   - Predictions range: {submission['prediction'].min():.4f} to {submission['prediction'].max():.4f}")
print(f"   - No NaN values: {submission['prediction'].isnull().sum() == 0}")

✅ Submission file saved as: submission.csv
📊 Submission stats:
   - Shape: (254569, 2)
   - Predictions range: 0.0001 to 1.0000
   - No NaN values: True


In [185]:
# Let's trace through your exact preprocessing pipeline
print("=== YOUR PREPROCESSING PIPELINE ===")

# 1. Check power transform
print("1. Power Transform Applied:")
print(f"   Train shape: {train_df.shape}")
print(f"   Test shape: {test.shape}")

# 2. Check dtype conversion
print("\n2. Data Type Conversion:")
print(f"   Train dtypes: {train_df_preproc.dtypes.value_counts()}")
print(f"   Test dtypes: {test_df_preproc.dtypes.value_counts()}")

# 3. Check label encoding
print("\n3. Label Encoding Check:")
for col in cat_features:
    if col in train.columns and col in test.columns:
        train_vals = sorted(train[col].unique())
        test_vals = sorted(test[col].unique())
        print(f"   {col}: train={train_vals}, test={test_vals}")
        
        # Check if any test values weren't seen in training
        unseen = set(test_vals) - set(train_vals)
        if unseen:
            print(f"   ❌ UNSEEN VALUES IN TEST: {unseen}")

# 4. Check scaling
print("\n4. Scaling Check:")
for col in num_features:
    if col in x_train.columns and col in test_df_preproc.columns:
        train_mean = x_train[col].mean()
        test_mean = test_df_preproc[col].mean()
        print(f"   {col}: train_mean={train_mean:.2f}, test_mean={test_mean:.2f}")

=== YOUR PREPROCESSING PIPELINE ===
1. Power Transform Applied:
   Train shape: (593994, 13)
   Test shape: (254569, 12)

2. Data Type Conversion:
   Train dtypes: int64      8
float64    5
Name: count, dtype: int64
   Test dtypes: float64    6
int64      6
Name: count, dtype: int64

3. Label Encoding Check:
   gender: train=[np.int64(0), np.int64(1), np.int64(2)], test=[np.int64(0), np.int64(1), np.int64(2)]
   marital_status: train=[np.int64(0), np.int64(1), np.int64(2), np.int64(3)], test=[np.int64(0), np.int64(1), np.int64(2), np.int64(3)]
   education_level: train=[np.int64(0), np.int64(1), np.int64(2), np.int64(3), np.int64(4)], test=[np.int64(0), np.int64(1), np.int64(2), np.int64(3), np.int64(4)]
   employment_status: train=[np.int64(0), np.int64(1), np.int64(2), np.int64(3), np.int64(4)], test=[np.int64(0), np.int64(1), np.int64(2), np.int64(3), np.int64(4)]
   loan_purpose: train=[np.int64(0), np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5), np.int64(6), np.in

In [186]:
from sklearn.model_selection import train_test_split

# Split your training data into train/test (not just train/val)
x_temp, x_test_sim, y_temp, y_test_sim = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Then split temp into train/val
x_train_sim, x_val_sim, y_train_sim, y_val_sim = train_test_split(
    x_temp, y_temp, test_size=0.25, random_state=42, stratify=y_temp  # 0.25 * 0.8 = 0.2 of original
)

print(f"Simulated train: {x_train_sim.shape[0]:,} samples")
print(f"Simulated val: {x_val_sim.shape[0]:,} samples") 
print(f"Simulated test: {x_test_sim.shape[0]:,} samples")

# Train your best model on simulated train
best_tuned.fit(x_train_sim, y_train_sim)

# Evaluate on simulated test (this mimics real test performance)
test_predictions = best_tuned.predict_proba(x_test_sim)[:, 1]
test_auc = roc_auc_score(y_test_sim, test_predictions)

print(f"\n🎯 SIMULATED TEST PERFORMANCE: {test_auc:.4f}")
print(f"📈 This should be close to your final Kaggle score")

Simulated train: 356,396 samples
Simulated val: 118,799 samples
Simulated test: 118,799 samples


KeyboardInterrupt: 